# Profit and Loss Historical Simulation

- Take the past 1 year worth of historical data,
- for each no-premium fee calculation (BSM, MC Normal, etc.),
- assume a 180 samples lookback fee calculation
- calculate the PnL of each period
- take a look at the Profit and Loss distribution and cummulative PnL over time
- evaluate the results
- compare with-fees vs no-fees

In [ ]:

import math
import random as random
import pandas as pd
import numpy as np
from numpy.random import default_rng
np.set_printoptions(precision=5, suppress=True)
# fixes the seed for reproducibility
rng = default_rng(99)
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as scipy
from scipy.optimize import fsolve
import psycopg2
# import seaborn as sns
from math import log, sqrt, exp, pi
from scipy.stats import norm, laplace, t as student_t
# Markdown in code cell
from IPython.display import Markdown as md
plt.style.use('seaborn')
plt.colormaps
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
# BSM Fee calculator
def bsm_no_premium_call_option_fee(S0, T, sigma):
    # BSM no-premium option strike calculator
    def bsm_no_premium_strike(S0, T, sigma):
        # init
        r = 0.0
        K_at_money = S0
        # bsm terms
        d1 = (log(S0/K_at_money) + (r + 0.5 * sigma**2) * T) / sigma / sqrt(T)
        d2 = d1 - sigma * sqrt(T)
        N_d1 = norm.cdf(d1)
        N_d2 = norm.cdf(d2)
        
        def bsm_premium(K):
            # bsm premium
            BSM_C0_no_premium = S0 * N_d1 - K * exp(-r*T) * N_d2
            return BSM_C0_no_premium
        
        K_no_premium = fsolve(bsm_premium, K_at_money)[0]
        
        return K_no_premium
    K_no_premium = bsm_no_premium_strike(S0, T, sigma)
    fee = (K_no_premium-S0)/S0
    return fee
# MC Normal Fee calculator
def mc_no_premium_call_option_fee(S0, T, sigma, I):
    # MC no-premium option strike calculator
    def mc_no_premium_strike(S0, T, sigma, I):
        # init
        r = 0.0
        K_at_money = S0
        # underlying price vector at maturity
        mc_rng = default_rng()
        ST = S0 * np.exp((r - sigma ** 2 / 2) * T + sigma * math.sqrt(T) * mc_rng.standard_normal(I))
        def mc_premium(K):
            # not exercised
            CT_below_K_at_money = ST[ST <= K_at_money] * 0
            # exercised at a loss
            CT_below_K = ST[ST > K_at_money]
            CT_below_K = CT_below_K[CT_below_K <= K] - K
            # exercised at a gain
            CT_above_K = ST[ST > K] - K
            # call option payoff at maturity
            CT_no_premium = np.concatenate((CT_below_K_at_money, CT_below_K, CT_above_K))
            CT_no_premium_mean = CT_no_premium.mean()
            # call option fair value
            MC_C0_no_premium = math.exp(-r * T) * CT_no_premium_mean
            return MC_C0_no_premium
        
        K_no_premium = fsolve(mc_premium, K_at_money)[0]
        
        return K_no_premium
    K_no_premium = mc_no_premium_strike(S0, T, sigma, I)
    fee = (K_no_premium-S0)/S0
    return fee
# MC Laplace Fee calculator
def mc_laplace_no_premium_call_option_fee(S0, T, loc, scale, I):
    # MC Laplace no-premium option strike calculator
    def mc_laplace_no_premium_strike(S0, T, loc, scale, I):
        # init
        r = 0.0
        K_at_money = S0
        # underlying price vector at maturity
        mc_rng = default_rng()
        ST = S0 * np.exp(laplace.rvs(loc, scale, size=I, random_state=99))
        def mc_premium(K):
            # not exercised
            CT_below_K_at_money = ST[ST <= K_at_money] * 0
            # exercised at a loss
            CT_below_K = ST[ST > K_at_money]
            CT_below_K = CT_below_K[CT_below_K <= K] - K
            # exercised at a gain
            CT_above_K = ST[ST > K] - K
            # call option payoff at maturity
            CT_no_premium = np.concatenate((CT_below_K_at_money, CT_below_K, CT_above_K))
            CT_no_premium_mean = CT_no_premium.mean()
            # call option fair value
            MC_C0_no_premium = math.exp(-r * T) * CT_no_premium_mean
            return MC_C0_no_premium
        
        K_no_premium = fsolve(mc_premium, K_at_money)[0]
        
        return K_no_premium
    K_no_premium = mc_laplace_no_premium_strike(S0, T, loc, scale, I)
    fee = (K_no_premium-S0)/S0
    return fee
# Data set
btcusd_1min = pd.read_csv("./BTCUSD.csv", sep=",", header=0, names=["date","open","high","low","close"], index_col="date")
btcusd_1min['ret_2min'] = np.log(btcusd_1min['close'] / btcusd_1min['close'].shift(2))
# Using the most recent 30 days worth of data only
nb_2min_in_30d = 30*24*30 # = 21600
ret_2min = btcusd_1min['ret_2min'][-21600:]
# ret_2min = btcusd_1min['ret_2min'][2:]
# the 2-min vol
vol_2min = ret_2min.std()
# number of 2-minute per year
nb_2min_per_year = 60/2 * 24 * 365.25
vol_ann = vol_2min * sqrt(nb_2min_per_year)
